# 2) Create runs.pkl file from optimisazion checkpoints

After running the model optimizations with the `run_optimizations.py` script, this notebook checks the optimization outputs and it creates a summary file called `runs.pkl` in the `optimization_results` folder.

The `runs.pkl` is used in notebooks 3-4 to evaluate and compare the optimization outputs.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import json
import glob

from pathlib import Path

import multimodalfitting as mf
from multimodalfitting.utils import load_checkpoint, read_checkpoint

%matplotlib notebook

In [ ]:
results_folder = Path('../optimization_results/checkpoints')

In [ ]:
runs = []
eva_json = []

for p in results_folder.iterdir():
    
    if "tmp" in p.suffix or "runs" in p.name or "json" in p.suffix:
        continue
    
    print(p)
    runs.append(load_checkpoint(p))
    eva_json.append(p.parent / f"{p.stem}.json")

In [ ]:
models = set([run["model"] for run in runs])
colors_model = {m: "C{}".format(i) for i,m in enumerate(models)}
print(colors_model)
colors_set = {"soma": "C0", "extra": "C1"} 

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

for run in runs:

    ax.plot(run["nevals"], 
            run["logbook"].select("min"),
            color=colors_model[run["model"]],
            ls='--', 
            lw=1,
            alpha=0.75,
            label=run["model"])
    
    ax.scatter([run["nevals"][-1]], 
               [np.sum(run["hof"][0].fitness.values)],
               color=colors_model[run["model"]],
               alpha=0.75)
    
ax.set_xlabel("Number of evaluations", size="x-large")
ax.set_ylabel("Minimum fitness (std)", size="x-large")

ax.set_yscale('log')
ax.legend(fontsize=15)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

for run in runs:

    ax.plot(run["nevals"], 
            run["logbook"].select("min"),
            color=colors_set[run["feature_set"]],
            ls='--', 
            lw=1,
            alpha=0.75,
            label=run["feature_set"])
    
    ax.scatter([run["nevals"][-1]], 
               [np.sum(run["hof"][0].fitness.values)],
               color=colors_set[run["feature_set"]],
               alpha=0.75)
    
ax.set_xlabel("Number of evaluations", size="x-large")
ax.set_ylabel("Minimum fitness (std)", size="x-large")

ax.set_yscale('log')
ax.legend(fontsize=15)

## Save results to pkl file

In [ ]:
with open(str(results_folder / "runs.pkl"), 'wb') as fp:
    pickle.dump(runs, fp)

## (optional) Check optimization responses

In [ ]:
for (run, eva) in zip(runs, eva_json):
    
    eva_args = json.load(eva.open("r"))
    evaluator = mf.create_evaluator(**eva_args)
    
    best_params = evaluator.param_dict(run["best_params"])
    responses = evaluator.run_protocols(protocols=evaluator.fitness_protocols.values(), 
                                        param_values=best_params)
    fig = mf.plot_responses(responses, return_fig=True)
    fig.subplots_adjust(top=0.85)
    fig.suptitle(eva.stem, y=0.95, fontsize=20)